In [ ]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


C:\Users\chimi\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\chimi\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\chimi\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\chimi\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\chimi\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary inco

### Objective:
- Objective : Developing a model to assess mortality risk based on individual characteristics and external variables is the primary goal of utilizing this final integrated information in order to compare and calculate insurance prices.


In [9]:
# Read the file 
df = pd.read_csv('C:/Users/chimi/Desktop/Python Data Science Projects/life_insurance/Untitled Folder/final_merged_df.csv')

In [10]:
# Check the shape of the file
print(f'Number of rows and columns: {df.shape[0]}, {df.shape[1]}')

Number of rows and columns: 76102, 22


In [11]:
df.head(5)

,ID,Gender,Birth_Date,Effective_Date,Capital,Renewal_Date,Age,t,Age_Actuarial,Birthday,...,s,Age_actuarial_quarter,Month,Year,Air Pollution Mortality_byGender,Poisoning Mortality_byGender,Adult Mortality_byGender,Infant Mortality_byGender,Under 5 Mortality_byGender,Suicide Mortality_byGender
0,1,M,1960-07-10,2000-03-08,55000.0,2009-03-08,48.659822,0.659822,49,2009-07-10,...,1,48.75,3,2000,NaN,0.6,118.721,4.7,5.9,13.2
1,2,M,1961-08-18,2000-03-07,105000.0,2009-03-07,47.550992,0.550992,48,2009-08-18,...,1,47.50,3,2000,NaN,0.6,118.721,4.7,5.9,13.2
2,3,M,1963-10-14,2000-03-15,79500.0,2009-03-15,45.418207,0.418207,45,2009-10-14,...,1,45.50,3,2000,NaN,0.6,118.721,4.7,5.9,13.2
3,4,F,1966-06-26,2000-03-15,74500.0,2009-03-15,42.718686,0.718686,43,2009-06-26,...,1,42.75,3,2000,NaN,0.3,47.685,4.0,4.9,4.2
4,5,M,1942-09-30,2000-03-07,140000.0,2009-03-07,66.433949,0.433949,66,2009-09-30,...,1,66.50,3,2000,NaN,0.6,118.721,4.7,5.9,13.2


In [12]:
df.columns

Index(['ID', 'Gender', 'Birth_Date', 'Effective_Date', 'Capital',
       'Renewal_Date', 'Age', 't', 'Age_Actuarial', 'Birthday', 'x', 'r', 's',
       'Age_actuarial_quarter', 'Month', 'Year',
       'Air Pollution Mortality_byGender', 'Poisoning Mortality_byGender',
       'Adult Mortality_byGender', 'Infant Mortality_byGender',
       'Under 5 Mortality_byGender', 'Suicide Mortality_byGender'],
      dtype='object')

In [13]:
# Check for duplicates
df.duplicated().sum()

0

In [14]:
# Check for missing values
df.isnull().sum()

ID                                      0
Gender                                  0
Birth_Date                              0
Effective_Date                          0
Capital                                 0
Renewal_Date                            0
Age                                     0
t                                       0
Age_Actuarial                           0
Birthday                                0
x                                       0
r                                       0
s                                       0
Age_actuarial_quarter                   0
Month                                   0
Year                                    0
Air Pollution Mortality_byGender    76102
Poisoning Mortality_byGender            0
Adult Mortality_byGender                0
Infant Mortality_byGender               0
Under 5 Mortality_byGender              0
Suicide Mortality_byGender              0
dtype: int64

In [15]:
# Drop the 'Air Pollution Mortality_byGender' 
df.drop('Air Pollution Mortality_byGender', axis=1, inplace=True)

#### Feature Selection:

In [16]:
# Drop data variables and the specified Air Pollution Mortality by Gender
df = df.drop(columns=['Birth_Date', 'Effective_Date', 'Renewal_Date','ID'])


In [17]:
# Exclude direct mortality measures to avoid data leakage (we want to predict mortality without using other mortality rates)
predictors_to_exclude = ['Infant Mortality_byGender', 'Under 5 Mortality_byGender', 'Suicide Mortality_byGender', 'Poisoning Mortality_byGender']


In [18]:
# Save the new changes to the dataframe
df = df.drop(columns=predictors_to_exclude)

In [19]:
# Check remaining columns for feature selection
remaining_cols = df.columns.tolist()
remaining_cols

['Gender',
 'Capital',
 'Age',
 't',
 'Age_Actuarial',
 'Birthday',
 'x',
 'r',
 's',
 'Age_actuarial_quarter',
 'Month',
 'Year',
 'Adult Mortality_byGender']

In [20]:
# # Check for multicollinearity between 'Age' and 'Age_Actuarial'
# Since, correlation between 'Age' and Age _Actuarial is extremely high 0.99, indicating multicolinearity if included
# Due to the high correlation and to avoid multicollinearity, we will just use the one of the variable for our predictive model
correlation = df[['Age', 'Age_Actuarial']].corr()

print(correlation)


                    Age  Age_Actuarial
Age            1.000000       0.999646
Age_Actuarial  0.999646       1.000000


#### Observation:
- For life insurance, the 'Age_Actuarial' might be more suitable since it rounded to whole years,a common practice among actuarial calculations for pricing

In [21]:
# Drop the Age_Actuarial column
df.drop(columns=['Age'], axis=1, inplace=True)

#### Encoding Gender variable

In [96]:
# Encode the Gender
df['Gender'] = df['Gender'].map({'M':1, 'F': 0})

In [97]:
# Check the first fews rows to confirm encoding 
df.head(5)

,Gender,Capital,t,Age_Actuarial,Birthday,x,r,s,Age_actuarial_quarter,Month,Year,Adult Mortality_byGender
0,1,55000.0,0.659822,49,2009-07-10,0.180822,4,1,48.75,3,2000,118.721
1,1,105000.0,0.550992,48,2009-08-18,0.178082,3,1,47.50,3,2000,118.721
2,1,79500.0,0.418207,45,2009-10-14,0.200000,3,1,45.50,3,2000,118.721
3,0,74500.0,0.718686,43,2009-06-26,0.200000,4,1,42.75,3,2000,47.685
4,1,140000.0,0.433949,66,2009-09-30,0.178082,3,1,66.50,3,2000,118.721


In [98]:
df['Gender'].value_counts()

Gender
1    47652
0    28450
Name: count, dtype: int64

### Feature Selection Part 2:

In [101]:
# 
df.drop(columns=['Birthday','Year'],  inplace=True)

#### Drop 't', 'x', 'r', 's' as we have no way to say it shared a significant relationship with the target variable
- Focusing on variables that have the most direct relationship to mortaabslity

#### Build a baseline modeling using Linear Regression
- There are several benefits using Linear Regression to assess mortality risk based on individual characteristics and external variables.Using Linear Regression to forecast mortality allows us to adjust cpaital to make premium comuptations more precise and ensures it closely matches premium to individual risk profiles. Also, it allows insurance companies to allocate capital accurately, improving overall risk management. Additionally, it help us create life insurance polcies that can meet the demand of individual clients, boosting the policyholders satisfaction and giving a business more competetive edge.
- After creating a baseline model using Linear Regression, we will use those mortality predictions to categorize the different types of premium that can be offered to policyholders.

In [110]:


# Prepare the dataset for Linear Regression with the selected predictors 
selected_predictors = ['Gender', 'Capital', 'Age_Actuarial']
X = df[selected_predictors]
y = df['Adult Mortality_byGender']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model on the selected predictors
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the test set and evaluate performance metrics
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r_square = r2_score(y_test, y_pred)

# Print the performance metrics
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared:", r_square)

Mean Squared Error (MSE): 37.200864722148616
Root Mean Squared Error (RMSE): 6.099251160769543
R-squared: 0.9562793476111063


In [111]:
# Use the predicted values to perform risk segementation
y_all_pred = model.predict(X)

pd.Series(y_all_pred).describe()

count    76102.000000
mean        81.097226
std         28.578519
min         40.574599
25%         44.460453
50%        102.588755
75%        103.399630
max        105.548738
dtype: float64

### Observation: 
- The minimum is 40.60, 25th Percentile is 44.46, Median is (50th Percentile): 102. 59, 75th Percentile:103.40, and Maximum 105.55

In [114]:
# Categorize individuals into risk segments based on the defined thresholds
def categorize_risk(rate):
    if rate <= 102.59:  # 50th percentile
        return 'Low Risk'
    elif rate <= 103.40:  # 75th percentile
        return 'Medium Risk'
    else:
        return 'High Risk'

risk_categories = pd.Series(y_all_pred).apply(categorize_risk)

# Count the number of individuals in each risk segment
risk_segment_counts = risk_categories.value_counts()

risk_segment_counts

Low Risk       38084
Medium Risk    19042
High Risk      18976
Name: count, dtype: int64

In [115]:

expected_claim_amount = 1000
claim_frequency_low = 0.01  # 1%
expenses_rate = 0.20  # 20%
profit_margin = 0.10  # 10%

# Calculate Base Premium for Low Risk
base_premium_low = (expected_claim_amount / claim_frequency_low) * (1 + expenses_rate + profit_margin)

# Adjust for Medium and High Risk
multiplier_medium = 1.5  # 50% higher risk than low
multiplier_high = 2.0  # 100% higher risk than low

premium_medium = base_premium_low * multiplier_medium
premium_high = base_premium_low * multiplier_high

base_premium_low, premium_medium, premium_high

(130000.0, 195000.0, 260000.0)

In [116]:
X

,Gender,Capital,Age_Actuarial
0,1,55000.0,49
1,1,105000.0,48
2,1,79500.0,45
3,0,74500.0,43
4,1,140000.0,66
...,...,...,...
76097,0,95000.0,45
76098,0,45000.0,44
76099,0,103000.0,40
76100,0,82000.0,43
